# Section D1 - Pandas Example

Let's do some simple data analysis and presentatin with some plots.  Nasa has this list of meteorites that we can use.  It doesn't require much cleaning/prep to use, so shouldn't take too many steps to make some nice plots and make some observtions.

First thing is importing.  We use requests to query the url, get the json data, and convert it to a dataframe.  A few useful funtions for viewing data in a dataframe are .head(), .tail(), and .info().  

There's more info on this dataset here: https://data.nasa.gov/Space-Science/Meteorite-Landings/gh4g-9sfh/about_data

In [ ]:
import requests
import pandas as pd
meteorites = requests.get('https://data.nasa.gov/resource/y77d-th95.json').json()
mets = pd.DataFrame(meteorites)
mets.head()

Great, so we have geolocation, mass, and datetime info for each meteor.  Let's try graphing mass per date to see if there's any obvious trend. 

We need each column to be the correct datatype before we can generate a plot.
* To simplify the datetime 'year' column, I use a string operation to split it on the 'T' ang take just the year, month and day.  Then we can use pd.to_datetime do convert it to a datetime object by passing in the format to use to convert it. 
* We need the mass to be a numeric value so we overwrite the column with itself converted using pd.to_numeric.  Similarly, there ar pd.to_int, pd.to_float, pd.to_string operatoins that we might want to use in other cases. 

Finally, pandas has a built in plot function that can generate a bunch of different graph types. Setting 'logy' says to graph the y axis in log scale. Try setting it and see what happens to the data points and y axis scale. 

https://www.w3schools.com/python/gloss_python_date_format_codes.asp

In [ ]:
# Graph the meteorites by date and mass
mets['ymd'] = mets['year'].str.split('T').str[0]
mets['ymd'] = pd.to_datetime(mets['ymd'], format='%Y-%m-%d', errors='coerce')
mets['mass'] = pd.to_numeric(mets['mass'])
mets.plot.scatter(x='ymd', y='mass', logy=True, title='Meteorite Mass by Date')

That's neat, but doesn't show much of a trend except that we probably have better records of meteorites found since the late 1800s.  Maybe it would be interesting to see where on earth we are finding the meteorites.  

Let's plot them on a map of the earth.  First thing for that is to get a map of the earth.  We can use some geopandas stuff for that.  Below, "world" is a dataframe with rows for each landmass on a map.  Try printing world.head() to see some of the actual data.  

https://geopandas.org/en/stable/docs/user_guide.html

In [ ]:
import geopandas
from geodatasets import get_path

path = get_path("naturalearth.land")
world = geopandas.read_file(path)
world.plot()

And now lets put it together!  Plotting can get complicated fast... I think that's a weakness of this stuff in python with the pandas, matplotlib libraries, but it's very powerful at least. 

To combine plots, we initialize an axis that we pass to the .plot function when we call it for the world and our meteorite dataframes so that the can draw themselves on the same graph. 'ax=ax' looks a littele weird.  We'r passing a variable named ax to an argument with the same name.  It's just sort of convention to do it this way.  Maybe it would be better to use axis for the variable name and pass that to the plot function. 

We need to convert our reclong and reclat (longitude and latitude) to numeric values to plot them, so use call .astype(float) to do a type conversion from string.

You can change the colormap - if you put in a bad value, it'll print a bunch you can try in the error message.  And the norm= is to convert the mass to log scale here so that we get nice colors for all of the meteorite masses. 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12,6))           # create a figure and axis

world.plot(ax=ax)                                # plot the world on the axis

mets['reclong'] = mets['reclong'].astype(float)  # convert reclong column from string to float
mets['reclat'] = mets['reclat'].astype(float)    # convert reclat column from string to float

# plot the meteorites on the same axis
mets.plot(x="reclong", y="reclat", kind="scatter", 
        c="mass", colormap="Accent_r", 
        title=f"Meteors around the world!", 
        ax=ax, norm=matplotlib.colors.LogNorm())

ax.grid(True)                                   # turn on the grid
plt.show()

Well that's neat.  I extected to see more concentration around the equator.  We might need to normalize for population density (people who could see and find an asteroid to report on) to get an idea of were most asteroids actually fall.  There is a larger version of this dataset at: 

We'll look at more actual numerical analysis stuff in Week 4.

#### *Exercise*
Open a new notebook and download the dataset for observed meteors from here: https://www.kaggle.com/datasets/ramjasmaurya/fireballsbolides-1988-april-2022.  Save it in the same directory as your notebook.  If you're working in Google Colab, you should be able to go to File -> Locate notebook in Drive, and then upload the dataset csv file to the same directory in Google drive. 

Copy over the following code to get started, and use the above example for the meteorites to make a couple of graphs for this new dataset.
* radiated energy vs time
* altitude vs radiated energy
* try using .corr(numeric_only=True) on the dataframe to see which numeric columns have the strongest correlation.   What can we say about these boloids when they enter our atmosphere?

In [ ]:
csv_file_name = 'nasas fireballs.csv'
nfb = pd.read_csv(csv_file_name)
# nfb.head() # Uncomment to see the first few rows of the data
# nfb.info() # Uncomment to see the column names and data types of each column

# the date/time ... column is a string that we want to convert to a datetime object
# we're creating a 'date' column for this.  We could also overwrite the existing column
# nfb['date'] = pd.to_datetime(nfb['date/time for peak brightness'])


#### *Exercise*
Go through some of these datasets and find something that looks interesting to you that we can work on next week.

* https://github.com/jdorfman/awesome-json-datasets - we can direcly query these using requests and the url as we did for the meteorite data. 
* https://catalog.data.gov/dataset/
* https://data.fivethirtyeight.com/ - they have zip files with csv data
* https://www.kaggle.com/datasets - click all data sets and you'll see loads of stuff.  looks like they have large csv files to download.